In [1]:
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import math
import random
from decimal import Decimal, getcontext
import pandas_market_calendars as mcal
import ast
from sklearn.preprocessing import StandardScaler
import pytz

In [2]:
s3 = boto3.client('s3')

training_bucket = "icarus-research-data"
training_prefix = 'training_datasets/expanded_1d_datasets/'

# Create a calendar
nyse = mcal.get_calendar('NYSE')
holidays = nyse.regular_holidays
market_holidays = holidays.holidays()

index = ["QQQ","SPY","IWM","TLT"]
test_lag = ["DAL","UAL","VZ","T","AAL","AMC"]
train_lag = ["AMC"]

In [3]:
def create_training_data_local(key_list, prefix, bucket_name, start_date, end_date):
    df_list = []
    hours = [10,11,12,13,14,15]
    start = start_date.split(' ')[0]
    end = end_date.split(' ')[0]
    # print(file_key)
    for key in key_list:
        print(key)
        for hour in hours:
            try:
                data = s3.get_object(Bucket=bucket_name, Key=f'{prefix}{key}/{hour}.csv')
                df = pd.read_csv(data.get("Body")) 
                df_list.append(df)
                df['hour'] = hour
            except:
                continue

    data = pd.concat(df_list)
    data.reset_index(drop=True, inplace=True)
    data['dt'] = pd.to_datetime(data['date'])
    data['day_of_week'] = data['dt'].apply(lambda x: x.dayofweek)
    data = data.round(3)
    data.replace([np.inf, -np.inf], 0, inplace=True)

    data.to_csv(f'/Users/charlesmiller/Documents/BF/{start}_{end}.csv', index=False)

    return data

def pull_training_data_local(end_date,start_date):
    data = pd.read_csv(f'/Users/charlesmiller/Documents/SF/2018-01-01_2023-10-12.csv')
    data['dt'] = pd.to_datetime(data['date'])
    data = data.loc[data['dt'] <= end_date]
    # data = data.loc[data['dt'] >= start_date]
    data = data.loc[~data['symbol'].isin(index)]
    data = data.loc[~data['symbol'].isin(train_lag)]
    data['roc_diff'] = data['roc'] - data['roc5']
    data['range_vol_diff5'] = (data['range_vol'] - data['range_vol5MA'])
    data['close_diff_deviation3'] = abs(data['close_diff3'])/(data['threeD_stddev50']*100)
    data['close_diff_deviation'] = abs(data['close_diff'])/(data['oneD_stddev50']*100)
    data['day_of_week'] = data['dt'].apply(lambda x: x.dayofweek).astype(int)
    data['day_of_month'] = data['dt'].apply(lambda x: x.day).astype(int)
    data['month'] = data['dt'].apply(lambda x: x.month).astype(int)
    data['year'] = data['dt'].apply(lambda x: x.year).astype(int)
    data.replace([np.inf, -np.inf], 0, inplace=True)

    return data


def pull_validation_data_local(end_date,start_date):
    data = pd.read_csv(f'/Users/charlesmiller/Documents/SF/2018-01-01_2023-10-12.csv')
    data['dt'] = pd.to_datetime(data['date'])
    data = data.loc[data['dt'] <= end_date]
    data = data.loc[data['dt'] >= start_date]
    data = data.loc[~data['symbol'].isin(index)]
    data = data.loc[~data['symbol'].isin(test_lag)]
    data['roc_diff'] = data['roc'] - data['roc5']
    data['range_vol_diff5'] = (data['range_vol'] - data['range_vol5MA'])
    data['close_diff_deviation3'] = abs(data['close_diff3'])/(data['threeD_stddev50']*100)
    data['close_diff_deviation'] = abs(data['close_diff'])/(data['oneD_stddev50']*100)
    data['day_of_week'] = data['dt'].apply(lambda x: x.dayofweek).astype(int)
    data['day_of_month'] = data['dt'].apply(lambda x: x.day).astype(int)
    data['month'] = data['dt'].apply(lambda x: x.month).astype(int)
    data['year'] = data['dt'].apply(lambda x: x.year).astype(int)
    data.replace([np.inf, -np.inf], 0, inplace=True)

    return data

In [4]:
def build_date_list(start_date, end_date):
    date_diff = end_date - start_date
    numdays = date_diff.days 
    dateList = []
    for x in range (0, numdays):
        temp_date = start_date + timedelta(days = x)
        if temp_date.weekday() > 4:
            continue
        else:
            dateList.append(temp_date)
    return dateList

def build_query_keys_hist():
    start_date = datetime(2021,1,5)
    date_diff = datetime(2022,7,29) - start_date
    numdays = date_diff.days 
    key_list = []
    for x in range (0, numdays):
        temp_date = start_date + timedelta(days = x)
        if temp_date.weekday() > 4:
            continue
        else:
            date_str = temp_date.strftime('%Y-%m-%d')
            if date_str in market_holidays:
                continue
            else:
                date_str = date_str.replace("-","/")
                key_list.append(date_str)
        
    return key_list
    
def build_query_keys(dates):
    key_list = []
    for date in dates:
        date_str = date.strftime('%Y-%m-%d')
        if date_str in market_holidays:
            continue
        else:
            year, month, day = date_str.split('-')
            temp = f'{year}/{month}/{day}'
            key_list.append(temp)

    return key_list

def build_query_keys_validation(end_date):
    validation_end_date = end_date + timedelta(days=7)
    dates = build_date_list(end_date, validation_end_date)
    key_list = []
    for date in dates:
        date_str = date.strftime('%Y-%m-%d')
        if date_str in market_holidays:
            continue
        else:
            year, month, day = date_str.split('-')
            temp = f'{year}/{month}/{day}'
            key_list.append(temp)

    return key_list

def build_validation_dates_local(deployment_date):
    end_date = deployment_date + timedelta(days=5)
    return end_date

In [5]:
def model_results_analyzer(predictions, y_validate, target_value):
    result_list = []
    counter = 0
    predictions_series = pd.Series(predictions,name='prediction_values')
    for x in predictions:
        if x == 1:
            if y_validate.iloc[counter] == 1:
                classification_result = 0
            else:
                classification_result = 1
        elif x == 0:
            if y_validate.iloc[counter] == 0:
                classification_result = 2
            else: 
                classification_result = 3
        result_list.append(classification_result)
        counter += 1
    three_max = pd.Series(y_validate,name='three_max')
    df = pd.concat([pd.Series(result_list,name='classifier_performance'),predictions_series,three_max],axis=1)
    df.reset_index(drop=True, inplace=True)
    # df = pd.DataFrame([result_list, y_validate], columns=['classifier_performance', 'prediction_score'])

    tp = df.loc[df['classifier_performance'] == 0]
    fp = df.loc[df['classifier_performance'] == 1]
    tn = df.loc[df['classifier_performance'] == 2]
    fn = df.loc[df['classifier_performance'] == 3]

    # tp_scr = tp["prediction_score"].mean()
    # fp_scr = fp["prediction_score"].mean()
    # tn_scr = tn["prediction_score"].mean()
    # fn_scr = fn["prediction_score"].mean()

    return len(tp), "0", len(fp), "0", len(tn), "0", len(fn), "0"

In [6]:
def create_dynamo_record(tp, tp_scr, fp, fp_scr, tn, tn_scr, fn, fn_scr, model_name, deployment_date, dataset_name, hyperparam_str, feature_str, target_str, fi_list):    
    ddb = boto3.resource('dynamodb','us-east-1')
    table = ddb.Table('icarus-models-results-table')
    eval_start = deployment_date
    eval_end = deployment_date + timedelta(days=4)
    try:
        precision = (tp/fp)
    except:
        precision = 0

    ## FILL IN
    item={
        'model_name': model_name,
        'deployment_date': deployment_date.strftime("%Y-%m-%d"),
        'algorithm_type': 'xgboost',
        'dataset': dataset_name,
        'TP': tp,
        'TPpct': Decimal("0"),
        'FP': fp,
        'FPpct': Decimal("0"),
        'TN': tn,
        'TNpct': Decimal("0"),
        'FN': fn,
        'FNpct': Decimal("0"),
        'precision_ratio': Decimal(str(precision)),
        'evaluation_timeperiod': f'{eval_start.strftime("%Y-%m-%d")}_{eval_end.strftime("%Y-%m-%d")}',
        'live': False,
        'hyperparameters': {hyperparam_str},
        'features' : {feature_str},
        'target' : target_str,
        'feature_importances': fi_list

    }

    print(item)
    response = table.put_item(
            Item=item
        )

    return response

In [7]:
def train_model(features, dataset, validation_dataset, target_label, target_value, hyperparams):
    dataset.loc[:, 'label'] = (dataset[target_label] > target_value).astype(int)
    validation_dataset.loc[:, 'label'] = (validation_dataset[target_label] > target_value).astype(int)

    dataset = dataset.round(3)
    validation_dataset = validation_dataset.round(3)


    X = dataset[features]
    y = dataset['label']

    X_validate = validation_dataset[features]
    y_validate = validation_dataset['label']

    print(y.value_counts())
    xgb_model = xgb.XGBClassifier(subsample=hyperparams['subsample'],num_round=hyperparams['num_round'],min_child_weight=hyperparams['min_child_weight'],max_depth=hyperparams['max_depth'],learning_rate=hyperparams['learning_rate'],gamma=hyperparams['gamma'],colsample_bytree=hyperparams['colsample_bytree'],verbosity=0,objective='binary:logistic',random_state=42)
    xgb_model.fit(X,y)

    predictions = xgb_model.predict(X_validate)
    probabilities = xgb_model.predict_proba(X_validate)
    tp, tp_scr, fp, fp_scr, tn, tn_scr, fn, fn_scr = model_results_analyzer(predictions, y_validate, target_value)

    fi = xgb_model.feature_importances_
    fi_list = []
    counter = 0
    for x in features:
        fi_list.append({x:fi[counter]})
        counter += 1
    print(tp,fp,tn,fn)
    return tp, tp_scr, fp, fp_scr, tn, tn_scr, fn, fn_scr,str(fi_list), predictions, probabilities


def train_model_TSSim(features, dataset, validation_dataset, target_label, target_value, hyperparams):
    dataset.loc[:, 'label'] = (dataset[target_label] > target_value).astype(int)
    validation_dataset.loc[:, 'label'] = (validation_dataset[target_label] > target_value).astype(int)

    dataset = dataset.round(3)
    validation_dataset = validation_dataset.round(3)


    X = dataset[features]
    y = dataset['label']

    X_validate = validation_dataset[features]
    y_validate = validation_dataset['label']

    xgb_model = xgb.XGBClassifier(subsample=hyperparams['subsample'],num_round=hyperparams['num_round'],min_child_weight=hyperparams['min_child_weight'],max_depth=hyperparams['max_depth'],learning_rate=hyperparams['learning_rate'],gamma=hyperparams['gamma'],colsample_bytree=hyperparams['colsample_bytree'],verbosity=0,objective='binary:logistic',random_state=42)
    xgb_model.fit(X,y)

    predictions = xgb_model.predict(X_validate)
    tp, tp_scr, fp, fp_scr, tn, tn_scr, fn, fn_scr = model_results_analyzer(predictions, y_validate, target_value)

    return tp, tp_scr, fp, fp_scr, tn, tn_scr, fn, fn_scr,"str(fi_list)", predictions, "probabilities"

In [8]:
def model_runner_v2(model_name, dataset_name, title, features, target_label, target_value,start_date, end_date,deployment_date, feature_str, hyperparams_str, hyperparams,local_data, dataset_start_date):
    # dates = build_date_list(start_date, end_date)
    # key_list = build_query_keys(dates)
    validation_end_date = build_validation_dates_local(deployment_date)
    # validation_keys = build_query_keys_validation(deployment_date)
    # validation_dataset = create_validation_data(validation_keys, 'fixed_alerts_full/new_features/big_fish_stable/', 'inv-alerts')
    validation_dataset = pull_validation_data_local(validation_end_date,deployment_date)
    dataset = pull_training_data_local(end_date, dataset_start_date)
    # else:
    #     dataset = create_training_data_v2(key_list, 'fixed_alerts_full/new_features/big_fish/', 'inv-alerts')
    dataset.dropna(subset=["close_diff_deviation3"],inplace=True)
    validation_dataset.dropna(subset=["close_diff_deviation3"],inplace=True)
    tp, tp_scr, fp, fp_scr, tn, tn_scr, fn, fn_scr, fi_list, predictions, probabilities = train_model(features, dataset, validation_dataset, target_label, target_value, hyperparams)
    response = create_dynamo_record(tp, tp_scr, fp, fp_scr, tn, tn_scr, fn, fn_scr, model_name, deployment_date, dataset_name,hyperparams_str,feature_str, f"{target_value}+{target_label}", fi_list)
    validation_dataset['probabilities'] = probabilities[:,1]
    validation_dataset['predictions'] = predictions
    validation_csv = validation_dataset.to_csv()
    put_response = s3.put_object(Bucket="icarus-research-data", Key=f"backtesting_data/inv_alerts/{dataset_name}/{title}/{deployment_date.strftime('%Y-%m-%d')}.csv", Body=validation_csv)
    return response

def model_runner_data(start_date,end_date,):
    dates = build_date_list(start_date, end_date)
    key_list = build_query_keys(dates)
    print(key_list[-1])
    dataset = create_training_data_local(key_list, 'sf/vol/', 'inv-alerts', start_date.strftime('%Y-%m-%d %H:%M:%S'),end_date.strftime('%Y-%m-%d %H:%M:%S'))
    return dataset


def model_runner_temporal_simulation(features, target_label, target_value,dataset_start_date,end_date,deployment_date,hyperparams):
    validation_end_date = build_validation_dates_local(deployment_date)
    dataset = pull_training_data_local(end_date, dataset_start_date)
    validation_dataset = pull_validation_data_local(validation_end_date,deployment_date)
    dataset.dropna(subset=["close_diff_deviation3"],inplace=True)
    validation_dataset.dropna(subset=["close_diff_deviation3"],inplace=True)
    dataset.dropna(subset=["close_diff_deviation"],inplace=True)
    validation_dataset.dropna(subset=["close_diff_deviation"],inplace=True)
    tp, tp_scr, fp, fp_scr, tn, tn_scr, fn, fn_scr, fi_list, predictions, probabilities = train_model_TSSim(features, dataset, validation_dataset, target_label, target_value, hyperparams)
    return tp, fp, fn, tn 

In [9]:
def build_evaluation_period(eval_start, eval_end):
    dates_list = []
    while eval_start <= eval_end:
        date_object = {
            "deployment_date": eval_start,
            "dataset_end": eval_start - timedelta(days=10),
            "dataset_start": datetime(2020,1,1)
        }
        dates_list.append(date_object)
        eval_start += timedelta(days=7)
    return dates_list

In [10]:
#ma_cont_classifier_EXPvDiff_HistData
#ma_cont_classifier_EXPvDiff_HistData_NOVAL
#ma_cont_classifier_EXPvDiffDYNTUN_HistData_NOVAL
#ma_cont_classifier_EXPvDiffDYNTUN_HistData_NOVAL
model_name = 'SFC:3D_TSSIM1_expVol_custHyP15_2018'
dataset_name = '3D_TSSIM1_expVol_custHypP15_2018'
title = 'SFC_3D'
hyperparams = {'subsample': 0.6, 'num_round': 1000, 'min_child_weight': 10, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 1}
features = ['close_diff_deviation', 'vol7', 'daily_volume_vol_diff30', 'price_10DDiff', 'close_diff3', 'return_vol_30D', 'hour_vol_diff_pct', 'v_diff_pct', 'month', 'hour_volume_vol_diff', 
            'rsi5', 'range_vol_diff5', 'daily_vol_diff30', 'rsi', 'roc_diff', 'range_vol5MA', 'price_25MA', 'year', 'min_vol_diff', 'return_vol_5D', 'daily_vol_diff', 'price14', 'adx', 
            'volume_10MA',"SPY_diff","SPY_1D"]
target_label = 'three_max'
target_value = .045
dataset_start_date = datetime(2018,1,1,tzinfo=pytz.timezone('US/Eastern')) 

dates_list = build_evaluation_period(datetime(2022,10,24,tzinfo=pytz.timezone('US/Eastern')), datetime(2023,10,20,tzinfo=pytz.timezone('US/Eastern')))
# date = dates_list[-1]
# response = model_runner_data(start_date=dataset_start_date,end_date=datetime(2023,10,12,tzinfo=pytz.timezone('US/Eastern')))
for date in dates_list:
    print(date)
    response = model_runner_v2(model_name, dataset_name, title, features, target_label, target_value,"dataset_start_date",date['dataset_end'],date['deployment_date'], str(features),str(hyperparams), hyperparams, local_data=True, dataset_start_date=date['dataset_start'])
    
    

{'deployment_date': datetime.datetime(2022, 10, 24, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>), 'dataset_end': datetime.datetime(2022, 10, 14, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>), 'dataset_start': datetime.datetime(2020, 1, 1, 0, 0)}
label
0    220433
1    113426
Name: count, dtype: int64
362 350 939 539
{'model_name': 'SFC:3D_TSSIM1_expVol_custHyP15_2018', 'deployment_date': '2022-10-24', 'algorithm_type': 'xgboost', 'dataset': '3D_TSSIM1_expVol_custHypP15_2018', 'TP': 362, 'TPpct': Decimal('0'), 'FP': 350, 'FPpct': Decimal('0'), 'TN': 939, 'TNpct': Decimal('0'), 'FN': 539, 'FNpct': Decimal('0'), 'precision_ratio': Decimal('1.0342857142857143'), 'evaluation_timeperiod': '2022-10-24_2022-10-28', 'live': False, 'hyperparameters': {"{'subsample': 0.6, 'num_round': 1000, 'min_child_weight': 10, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 1}"}, 'features': {"['close_diff_deviation', 'vol7', 'daily_volume_vol_diff30',

ValueError: Data must be 1-dimensional, got ndarray of shape (0, 0) instead